In [3]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [4]:
def turn_to_url(season):
    return 'http://survivor.wikia.com/wiki/Survivor:_' + season.replace(' ','_')

def process(string):
    stripped = string.strip()
    return re.sub('[0-9]+', '', stripped)

In [5]:
seasons = ['Borneo', 'The Australian Outback', 'Africa', 'Marquesas', 'Thailand', 'The Amazon', 
           'Pearl Islands', 'All-Stars', 'Vanuatu', 'Palau', 'Guatemala', 'Panama', 
           'Cook Islands', 'Fiji', 'China', 'Micronesia', 'Gabon', 'Tocantins', 'Samoa', 
           'Heroes vs. Villains', 'Nicaragua', 'Redemption Island', 'South Pacific',
           'One World', 'Philippines', 'Caramoan', 'Cagayan', 'San Juan del Sur',
           'Worlds Apart', 'Cambodia', 'Kaôh Rōng', 'Millennials vs. Gen X',
           'Game Changers', 'Heroes vs. Healers vs. Hustlers', 'Blood vs. Water']

# scrape data from web

In [204]:
data = []
for season in seasons:
    print(season)
    cara = requests.get(turn_to_url(season))
    soup = BeautifulSoup(cara.content, 'lxml')
    summary_table = soup.select('table.wikitable.article-table')[0]
    skip = 2
    if season == 'Gabon':
        skip = 3
    lastdat = -2
    if season in ['Redemption Island']:
        lastdat = -3
    num_contestants = len(summary_table.select('tr'))-skip
    for i in range(num_contestants):
        if len(summary_table.select('tr')[i+skip].select('td')) in [1, 2]:
            continue
        row_i = summary_table.select('tr')[i+skip]
        name = row_i.select('td b a')[0].text  
        ageloc = row_i.select('td > small')[0].text
        daysplacing = row_i.select('td')[lastdat].text
        data.append({'season': season, 'name': name, 'ageloc': ageloc, 'daysplacing': daysplacing})

Borneo
The Australian Outback
Africa
Marquesas
Thailand
The Amazon
Pearl Islands
All-Stars
Vanuatu
Palau
Guatemala
Panama
Cook Islands
Fiji
China
Micronesia
Gabon
Tocantins
Samoa
Heroes vs. Villains
Nicaragua
Redemption Island
South Pacific
One World
Philippines
Caramoan
Cagayan
San Juan del Sur
Worlds Apart
Cambodia
Kaôh Rōng
Millennials vs. Gen X
Game Changers
Heroes vs. Healers vs. Hustlers
Blood vs. Water


IndexError: list index out of range

In [96]:
rr = pd.DataFrame(data)

In [97]:
rr['age'] = rr['ageloc'].str.split(',').map(lambda x: x[0])

In [98]:
rr['state'] = rr['ageloc'].str.split(',').map(lambda x: x[2][:3])

In [ ]:
def check(row):
    x = row['daysplacing']
    last = ['Runner-Up', 'Sole Survivor', 'Runners-Up']
    for censored in last:
        if censored in x:
            if row['season'] == 'The Australian Outback':
                return 42
            else:
                return 39
    else:
        return x[x.find('Day')+4:x.find('Day')+6].strip()

In [99]:
rr['elim'] = rr.apply(check, axis=1)

In [101]:
rr['stripped'] = rr['daysplacing'].str.split('\n')

In [102]:
rr['is_censored'] = ~rr['daysplacing'].str.contains('Voted Out')

In [ ]:
rr['is_censored'] = rr['is_censored'].astype(int)

In [117]:
rv = rr[['name', 'season', 'age', 'state', 'elim', 'is_censored']]

# fix wrong entries

In [194]:
rv.iloc[94,3] = 'DC' # Matthew Amazon
rv.iloc[107,4] = 36 # Burton PI
rv.iloc[110,4] = 39
rv.iloc[110,5] = 1 # Lil PI

# redemption
rv.iloc[385,4] = 38
rv.iloc[385,5] = 0
rv.iloc[386,4] = 39
rv.iloc[387,4] = 39
rv.iloc[388,4] = 39

#south pacific
rv.iloc[389:404,4] = [3,8,11,14,16,5,24,21,27,27,30,32,35,37,38]
rv.iloc[389:404,5] = 0

#michael skupin
rv.iloc[441,4] = 39
#sheri
rv.iloc[461,4] = 39
#will sims
rv.iloc[515,4] = 39
#tasha
rv.iloc[535,4] = 39
#ken
rv.iloc[573,4] = 39

rv['state'] = rv['state'].str.strip()
rv['state'] = rv['state'].map(lambda x: 'DC' if x == 'D.' else x)

/home/pcalderon/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [208]:
rv.to_csv('castaways.csv', index=False)

# after labelling male and female, divide into groups based on:
## 1. geographic: east coast vs west coast vs no coast
## 2. geographic: northeast vs midwest vs south vs west
## 3. age group: young vs old
## 4. old school vs new school: age group and sex

In [10]:
west = ['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM']
midwest = ['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH']
south = ['OK', 'TX', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'WV', 'DC', 'MD', 'DE', 'VA', 'NC', 'SC', 'GA', 'FL']
northeast = ['PA', 'NY', 'VT', 'NH', 'ME', 'MA', 'CT', 'RI', 'NJ']

In [18]:
eastcoast = ['ME', 'NH', 'MA', 'RI', 'CT', 'NY', 'NJ', 'DE', 'ML', 'VA', 'NC', 'SC', 'GA', 'FL']
westcoast = ['CA', 'OR', 'WA']

In [34]:
ca = pd.read_csv('castaways.csv')

In [35]:
def map_state_to_region(state):
    if state in west:
        return 'West'
    elif state in midwest:
        return 'Midwest'
    elif state in south:
        return 'South'
    elif state in northeast:
        return 'Northeast'
    else:
        return 'NULL'

In [36]:
def map_state_to_coast(state):
    if state in eastcoast:
        return 'East Coast'
    elif state in westcoast:
        return 'West Coast'
    else:
        return 'Not Coast'

In [37]:
def map_season_to_school(season):
    if season in seasons[:18]:
        return 'Old School'
    else:
        return 'New School'

In [56]:
def map_age_group(age):
    if age <= 35:
        return 'Young'
    elif age <= 50:
        return 'Middle'
    else:
        return 'Old'

In [39]:
ca['region'] = ca['state'].map(map_state_to_region)

In [40]:
ca['coast'] = ca['state'].map(map_state_to_coast)

In [41]:
ca['school'] = ca['season'].map(map_season_to_school)

In [57]:
ca['agegroup'] = ca['age'].map(map_age_group)

In [44]:
ca['is_male'] = ca['is_male'].map(lambda x: 'Male' if x == 1 else 'Female')

In [58]:
ca['agegroup'].value_counts()

Young     415
Middle    168
Old        50
Name: agegroup, dtype: int64

In [62]:
ca.to_csv('castaways2.csv', index=False)